### Generate Figures

In [1]:
import pandas as pd
import numpy as np
import statistics

In [2]:
def read_topology_data(filepath):
    
    """Reads topology data from a .top file.

    Args:
        filepath (str): Path to the .top file containing trajectory data.

    Returns:
        Nanostar strand length.
    """
    
    # Load your dataset
    df = pd.read_csv(filepath, delimiter = ' ', header=None, names = range(0, 3))

    # Remove the first row
    df = df.iloc[1:].reset_index(drop=True)    
    df = df.iloc[:, 0]

    #Calcs length of a strand
    strandLength = (df.size)/3

    #returns the strand length
    return strandLength

In [3]:
def read_trajectory(filepath):

    """
    
    Reads the csv file data

    Args:
        filename (str): the path to the file

    Returns:
        pd.DataFrame containing all the data

    """

    data =  pd.read_csv(filepath, usecols=range(3), sep=" ", header = None) 
    return data

In [4]:
def create_dataframes_by_timestamp(data):
    """Creates separate DataFrames for each timestamp in the data.

    Args:
        data (pd.DataFrame): Dataframe containing particle positions.

    Returns:
        list[pd.DataFrame]: List of DataFrames, each representing a single timestamp.
    """

    # print(data)
    #creating list to hold dataframes
    dataframes = []

    #counts the number of time stamps
    timestamp_counter = -1

    #creating lists for each column on the dataframe
    xrows, yrows, zrows = [], [], []
    
    skiplist = 0

    #
    for i in range(len(data)):

        comx = data.iloc[i, 0]

        if comx == "t":
            timestamp_counter += 1
            xrows.append([])
            yrows.append([])
            zrows.append([])
            continue
        
        if comx == "t" or comx == "b" or comx == "E":
            skiplist = i
        
        if skiplist != i:
            
            xrows[timestamp_counter].append(comx)
            yrows[timestamp_counter].append(comy := data.iloc[i, 1])
            zrows[timestamp_counter].append(comz := data.iloc[i, 2])

    for x, y, z in zip(xrows, yrows, zrows):
        dataframes.append(pd.DataFrame({"X": x, "Y": y, "Z": z}))

    return dataframes

In [5]:
def average_values(df1, df2, df3):
    
    avg_dfs = []
    for i in range(len(df1)):
        # print(df1[i])
        
        X_avg, Y_avg, Z_avg = [], [], []

        for j in range(len(df1[0])):

            X_avg.append(statistics.mean([float(df1[i].iloc[j, 0]), float(df2[i].iloc[j, 0]), float(df3[i].iloc[j, 0])]))
            Y_avg.append(statistics.mean([float(df1[i].iloc[j, 1]), float(df2[i].iloc[j, 1]), float(df3[i].iloc[j, 1])]))
            Z_avg.append(statistics.mean([float(df1[i].iloc[j, 2]), float(df2[i].iloc[j, 2]), float(df3[i].iloc[j, 2])]))
            
        avg_dfs.append(pd.DataFrame({"X": X_avg, "Y": Y_avg, "Z" : Z_avg}))
    
    return avg_dfs

In [6]:
def prep_trajectory_data(filepaths):

    """

    Creates a properly formatted dataframe with all 3 averaged trajectory datas in a single data frame

    Args:
        filepaths (list): list of filepaths --> SHOULD ONLY CONTAIN 3 FILEPATHS

    Returns:
        pd.DataFrame containing averaged positional values of all 3 trajectories
    
    """

    #create a list to store all the formatted trajectories
    trajectories = []
    
    #iterates through specific range to produce correct numbers corresponding to trail number
    for fp in filepaths:

        #adds the unformatted trajectory to a temporary variable
        temp_traj= read_trajectory(fp)

        #formats the trajectory and appends it to the list of trajectories
        trajectories.append(create_dataframes_by_timestamp(temp_traj))

    #calls function to create a df with averaged positional values
    avg_traj = average_values(trajectories[0], trajectories[1], trajectories[2])

    return avg_traj

In [7]:
def prep_traj_data(filepaths):

    """

    Creates a properly formatted dataframe with all 3 averaged trajectory datas in a single data frame

    Args:
        filepaths (list): list of filepaths --> SHOULD ONLY CONTAIN 3 FILEPATHS

    Returns:
        pd.DataFrame containing averaged positional values of all 3 trajectories
    
    """

    #create a list to store all the formatted trajectories
    

    #adds the unformatted trajectory to a temporary variable
    temp_traj = read_trajectory(filepaths)

    #formats the trajectory and appends it to the list of trajectories
    trajectories = create_dataframes_by_timestamp(temp_traj)

    #calls function to create a df with averaged positional values
    # avg_traj = average_values(trajectories[0], trajectories[1], trajectories[2])

    return trajectories

# Calculate Angles

In [8]:
def read_topology_data(filepath):
    
    """Reads topology data from a .top file.

    Args:
        filepath (str): Path to the .top file containing trajectory data.

    Returns:
        pd.DataFrame of the topology file
    """
    
    # Load your dataset
    df = pd.read_csv(filepath, header = None)
    
    # Remove the first row
    df = df.iloc[1:].reset_index(drop=True)    
    df = df.iloc[:, 0]


    #returns the formatted topology file
    return df

In [9]:
def find_arm_indecies(filepath, SE, armNum, SEspacer, coreSpacer):

    """
    Finds the index numbers that correspond to a single arm. Will append these indecies to core and end lists

    Args:
        filepath (str): Path to the .top file containing trajectory data.
        SE (int): The number of sticky ends on a strand
        armNum (int): The number of arms the nanostar has
        SEspacer (int): The number of nucleotides that make up the spacer before the SE
        starting_indecies (list)
        ending_indecies (list)

    Returns:
        core and end indeciex list
    """
    #reads in the topology data
    topology = read_topology_data(filepath)

    # print(topology)

    strandCount = 0
    index_count = 0

    #Calculate the length of the strand
    strandLength = (topology.size)/armNum
    # print(strandLength)
    seq_length = int(strandLength-SE-SEspacer-coreSpacer)/2

    i = 0

    #defines a single strand that makes up the double stranded arm
    s = 0

    #defines whether the end of the double stranded arm has been reached
    end = False


    #list holds all the indecies for the core and ends
    indecies = []

    #continues to loop
    while True:

        #appends each index, will start by appending an index of 0
        # print(topology.iloc[i])
        indecies.append(i)

        #if the stand number is even
        #checks for the start of a single strand
        if s%2 == 0:

            #single strands have a length of 15 nucleotides
            #this will find the ending of the single strand
            #seqlengh -1
            i+= 14

        #if instead at the end of the single strand
        elif end == False:

            #add 3 to account for the core spacers and to move to the start of the next single strand
            #that makes up the double stranded arm
            #coreSpacer + 1
            i+= 3
            

            #also sents end to TRUE since the end of the double strand is approaching
            end = True

        #when the end of the double stranded arm is reached (end = TRUE)
        else:

            #accounts for the spacer and the SE to go to the next strand
            #SE + SEspacer + 1
            i+= 8

            #sets end to FALSE again since a new strand has started
            end = False

        #exit condition
        if i >= topology.size:
            break

        #incremented each time an index is added
        s+= 1

    #creating core and end indecies list
    core_indecies = []
    end_indecies = []

    #appends the first element to the end_indecies list   
    end_indecies.append(indecies[0])

    #used to append every 2 elements from indecies into respective list
    loop = 1

    #round - used to alternate appending to the end or core indecies lists
    rnd = 0

    #continues to run
    while True:

        #if the loop index exceeds the bound for the index
        if (loop + 2) >= len(indecies):
            #break the while loop
            break

        #if the round is even
        if rnd % 2 == 0:
            #append the indecies to core_indecies list
            core_indecies.append((indecies[loop], indecies[loop+1]))

        #if the round is odd
        else:
            #append the idencies to end_indecies list
            end_indecies.append(indecies[loop+1])

        #increment the loop by 2 to account for appending every 2 elements to a different list
        loop += 2
        #increment the loop by 1 to do binary list selection
        rnd +=1

    

    avg_core = []

    #Take averages of the core index pairs
    for pair in core_indecies:
        avg_core.append(int(statistics.mean(pair)))

    #

    return avg_core, end_indecies


In [10]:
# find_arm_indecies("/Users/pradnyakadam/Downloads/ACS_Manuscript/5_arm_INPUT_FILES/5m_6NT_2bp.top", 6, 5, 1, 2)

In [11]:
def core_NTs(dfs, SE, armNum, seq_len, core_len, SEspacer):

    """
    Calculates the indecies of the core nucleotides

    Args:
        dfs (df of dfs): data frame containing dataframes of nucleotide positions corresponding to each time point
        SE (int): the number of sticky ends in a strand
        seq_len (int): the length of a sequence
        core_len (int): the number of core nucleotides in a strand
        SEspacer (int): the number of sticky end spacers on a strand

    Returns:
        The core indecies
        
    """
    
    #list stores core NT line #s
    core = []
    
    jump = 0
    
    #calcs the # of NTs in each arm, excluding SE + spacer
    arm_len = (seq_len - core_len - SEspacer - SE)/2
    
    for c in range(0, arms):
        
        line = int(arm_len) + jump
        
        core.append(int(line))
        
        
        core.append(int(line + 1))
        
        jump += seq_len
        
    return core

In [12]:
def find_center_of_mass(dataframes, core_indecies):
    """Calculates the average center of mass for core nucleotides at each timestamp.

    Args:
        dataframes (list[pd.DataFrame]): List of DataFrames containing particle positions.
        core_nucleotides (int): Number of core nucleotides in the nanostar.
        arms (int): Number of arms in the nanostar.
        sequence_length (int): Total sequence length of the nanostar.

    Returns:
        pd.DataFrame: Dataframe containing average center of mass (COM) for each timestamp.
        Each row of the Dataframe corresponds to a new timestamp.
    """

    #creates a list to store average X, Y, and Z center of mass coordinates
    x_avg, y_avg, z_avg = [], [], []
    
    # arm_length = int((sequence_length - core_nucleotides) / (2 * armNum))

    #iterates through each time point's dataframe
    for df in dataframes:

        #creates lists to store the coordinates corresponding to each index in core_indecies list
        #stores the x, y, and z coordinates separately
        x_centers, y_centers, z_centers = [], [], []

        #iterates through the indecies in the core_indecies list
        for index in core_indecies:

            #adds the corresponding positional coordinate to its respective list
            x_centers.append(float(df.iloc[index, 0]))
            y_centers.append(float(df.iloc[index, 1]))
            z_centers.append(float(df.iloc[index, 2]))
                

        #averages the positional coordinates stored in the x, y, and z centers lists
        #this will provide the average center of mass at a single time point
        x_avg.append(statistics.mean(x_centers))
        y_avg.append(statistics.mean(y_centers))
        z_avg.append(statistics.mean(z_centers))

    return pd.DataFrame({"COM_X": x_avg, "COM_Y": y_avg, "COM_Z": z_avg})

In [13]:
def calculate_angle_between_arms(dataframes, com_dataframe, arm1_index, arm2_index):
    """Calculates the angle between two specified arms at each timestamp.

    Args:
        dataframes (list[pd.DataFrame]): List of DataFrames containing particle positions.
        com_dataframe (pd.DataFrame): Dataframe containing center of mass (COM) positions.


    Returns:
        list of angles formed by the selected arms over all the time points
    """

    #variable to iterate through each line of the center of mass df
    #as mentioned above each line of the com_dataframe represents a single timestamp of the simulation
    #this will ensure that each starting point is at the center of mass for the specific timestamp
    com_df = 0

    #list to hold all the angles created between two arms
    angles_list = []

    #iterates through each time point's dataframe
    for df in dataframes:

        #defines the starting point as the coordinates position associated with the particular timestamp
        #refernce comment above for more info.
        start_pt = np.array([com_dataframe.iloc[com_df, 0], com_dataframe.iloc[com_df, 1], com_dataframe.iloc[com_df, 2]])
#         print("start: ", start_pt)

        #defining vector 1 end point based on the provided index
        end_pt_1 = np.array([float(df.iloc[arm1_index, 0]), float(df.iloc[arm1_index, 1]), float(df.iloc[arm1_index, 2])])
#         print("ep1: ", end_pt_1)

        #defining vector 2 end point based on the other provided index
        end_pt_2 = np.array([float(df.iloc[arm2_index, 0]), float(df.iloc[arm2_index, 1]), float(df.iloc[arm2_index, 2])])
#         print("ep2: ", end_pt_2)

        #increment the com_df variable to go to the next timestamp's ceneter of mass
        com_df += 1

        #defining vectors 1 & 2 based on their corresponding end points
        v1 = end_pt_1 - start_pt
        v2 = end_pt_2 - start_pt

        #using the equation mathematical equation: 
        
        #calculating the dot product of the two vectors
        dot_product = np.dot(v1, v2)

        #calculating both vectors' magnitudes
        magnitude1 = np.linalg.norm(v1)
        magnitude2 = np.linalg.norm(v2)

        calc = dot_product / (magnitude1 * magnitude2)

        if calc > 1 or calc < -1:
            calc = 0

        #calculating the angle created between the two vectors (in radians)
        angle_radians = np.arccos(calc)

        #converting that value to degrees
        angle_degrees = np.degrees(angle_radians)
        
#         print("angle degrees: ", angle_degrees)

        #appending the calculated angle (based on the timestamp) to the list creeated above
        angles_list.append(angle_degrees)
        
#         print("done")
    
    return angles_list

# PUBLIC FUNCTION

In [14]:
def calc_angles(dataframes, topFile, SE, armNums, SEspacer, coreSpacer):

    """
    Returns the angles for each theta at all the different time points.

    Args:
        dataframes (list[pd.DataFrames]): List of DataFrames containing particle positions.

    Returns:
        list of angles for each theta value at each time point
    """

    avg_core_indecies, end_indecies = find_arm_indecies(topFile, SE, armNums, SEspacer, coreSpacer)
    com_dfs = find_center_of_mass(dataframes, avg_core_indecies)

    #theta1: angle btw arms 1 & 2
    #theta2: angle btw arms 2 & 3
    #theta3: angle btw arms 3 & 4
    #theta4: angle btw arms 1 & 4

    #NOTE THE INDECIES WILL CHANGE FOR DIFFERENT ARMED NANOSTAR

    #calculate_angle_between_arms(dataframes, com_dataframe, arm1_index, arm2_index)
    
    theta1 = calculate_angle_between_arms(dataframes, com_dfs, 0, 78)
    theta2 = calculate_angle_between_arms(dataframes, com_dfs, 78, 117)
    theta3 = calculate_angle_between_arms(dataframes, com_dfs, 117, 157)
    theta4 = calculate_angle_between_arms(dataframes, com_dfs, 157, 39)
    theta5 = calculate_angle_between_arms(dataframes, com_dfs, 39, 0)

    return pd.DataFrame({'θ1' : theta1,
                         'θ2' : theta2,
                         'θ3' : theta3,
                         'θ4' : theta4,
                         'θ5' : theta5})


# CALLS

## 5 arm

In [15]:
traj_65_1M = prep_trajectory_data(["/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_1.0M/trajectory_files/trajectory_65C_1.0M_sim_revised_1.dat",
                                   "/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_1.0M/trajectory_files/trajectory_65C_1.0M_sim_revised_2.dat",
                                   "/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_1.0M/trajectory_files/trajectory_65C_1.0M_sim_revised_3.dat"])

In [16]:
traj_65_75M = prep_trajectory_data(["/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_0.75M/trajectory_files/trajectory_65C_0.75M_sim_revised_1.dat",
                                   "/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_0.75M/trajectory_files/trajectory_65C_0.75M_sim_revised_2.dat",
                                   "/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_0.75M/trajectory_files/trajectory_65C_0.75M_sim_revised_3.dat"])

In [17]:
traj_65_5M = prep_trajectory_data(["/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_0.5M/trajectory_files/trajectory_65C_0.5M_sim_revised_1.dat",
                                   "/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_0.5M/trajectory_files/trajectory_65C_0.5M_sim_revised_2.dat",
                                   "/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_0.5M/trajectory_files/trajectory_65C_0.5M_sim_revised_3.dat"])

In [18]:
traj_65_25M = prep_trajectory_data(["/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_0.25M/trajectory_files/trajectory_65C_0.25M_sim_revised_1.dat",
                                   "/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_0.25M/trajectory_files/trajectory_65C_0.25M_sim_revised_2.dat",
                                   "/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_0.25M/trajectory_files/trajectory_65C_0.25M_sim_revised_3.dat"])

In [19]:
traj_65_01M = prep_trajectory_data(["/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_0.1M/trajectory_files/trajectory_65C_0.1M_sim_revised_1.dat",
                                   "/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_0.1M/trajectory_files/trajectory_65C_0.1M_sim_revised_2.dat",
                                   "/Users/pradnyakadam/Downloads/ACS_Manuscript/Simulation_Output_Files/5_Arm/65C_0.1M/trajectory_files/trajectory_65C_0.1M_sim_revised_3.dat"])

In [20]:
theta_65C_1M = calc_angles(traj_65_1M, "/Users/pradnyakadam/Downloads/ACS_Manuscript/5_arm_INPUT_FILES/5m_6NT_2bp.top", 6, 5, 1, 2)

In [21]:
theta_65C_75M = calc_angles(traj_65_75M, "/Users/pradnyakadam/Downloads/ACS_Manuscript/5_arm_INPUT_FILES/5m_6NT_2bp.top", 6, 5, 1, 2)

In [22]:
theta_65C_5M = calc_angles(traj_65_5M, "/Users/pradnyakadam/Downloads/ACS_Manuscript/5_arm_INPUT_FILES/5m_6NT_2bp.top", 6, 5, 1, 2)

In [23]:
theta_65C_25M = calc_angles(traj_65_25M, "/Users/pradnyakadam/Downloads/ACS_Manuscript/5_arm_INPUT_FILES/5m_6NT_2bp.top", 6, 5, 1, 2)

In [24]:
theta_65C_01M = calc_angles(traj_65_01M, "/Users/pradnyakadam/Downloads/ACS_Manuscript/5_arm_INPUT_FILES/5m_6NT_2bp.top", 6, 5, 1, 2)

# Plotting

In [25]:
def select_theta(theta):

    """
        Matches the inputted theta value to the corresponding bond angle.

    Args:
        theta (int): integer from 1 to 5 that corresponds to the specific bond angle within the nanostar conformation

    Returns:
        (string): the theta number
    """

    
    match theta:

        case 1: 
            return "θ1"
        case 2:
            return "θ2"
        case 3:
            return "θ3"
        case 4:
            return "θ4"
        case 5:
            return "θ5"
        

In [26]:
def legend_label(theta):

    """
        Matches the inputted theta value to the corresponding legend label for the graphs produced.

    Args:
        theta (int): integer from 1 to 5 that corresponds to the specific bond angle within the nanostar conformation

    Returns:
        (string): unicode letters for correct sub & superscripts
    """

    match theta:

        case 1: 
            return r'$\theta_{12}$'
        case 2:
            return r'$\theta_{23}$'
        case 3:
            return r'$\theta_{31}$'
        case 4:
            return r'$\theta_{41}$'
        case 5:
            return r'$\theta_{12}$'

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import statistics
import seaborn as sns
from scipy.interpolate import interp1d

In [28]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statistics
from scipy.interpolate import interp1d

# Seaborn style
sns.set(style="whitegrid")


def plot_interpolated_histogram(dfs, theta_value, condition_list, filename):
    """
    Produces an interpolated histogram plot for selected data points and prints statistics.

    Args:
        dfs (list[pd.DataFrame]): List of DataFrames containing angle data
        theta_value (int): the angle (between which arms) to plot
        condition_list (list): Conditions corresponding to each df in dfs
        filename (str): Filename to save the graph under

    Returns:
        A saved interpolated histogram figure
    """

    # Plotting setup
    plt.figure(figsize=(8, 5))
    # plt.rcParams['font.family'] = ['serif', 'Times New Roman']

    # creating lists to hold all statistical data
    mean = []
    stdev = []
    min_val = []
    max_val = []

    #keeps track of the index, and extracts the exact dataframe 
    for i, df in enumerate(dfs):
        
        # Extract relevant angle column
        angle_col = select_theta(theta_value)
        angles = df[[angle_col]]

        # Calculate statistics

        #gets rid of any NaN columns - extracts rest of the data
        data = angles[angle_col].dropna()

        #stats: mean, standard devication, min, max
        mean.append(data.mean())
        stdev.append(data.std())
        min_val.append(round(data.min(), 4))
        max_val.append(round(data.max(), 4))

        # Create histogram with forced range

        #Dividing the data into 6 equally spaced bins --> bins divided between the range of 1 and 180
        #bins: [0, 30, 60, 90, 120, 150, 180]
       
        #hist_counts: counts how many data points fall within a specified bin 
        #ex. if data was [15, 25, 40, 60, 70, 85, 95, 110, 150, 170], hist counts = [2, 1, 3, 2, 0, 2]
        hist_counts, bin_edges = np.histogram(data, bins=8, range=(0, 180))


        #finds the midpoint of each bin
        #will take each end of the bin and then finds the mindpoint
        #ex. for the bin: (0, 30] --> bin_center = 0+30/2 = 15
        #for an interpolated histogram, this shows you the data 
        bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

        #
        hist_frequencies = hist_counts / len(data)  # Normalize

        # Interpolation
        interpolator = interp1d(
            bin_centers, hist_frequencies, kind='quadratic', fill_value="extrapolate"
        )

        # will plot 30 points on each interpolated line
        x_dense = np.linspace(0, 180, 30)
        y_dense = np.clip(interpolator(x_dense), 0, None)  # Clamp negatives

        # Plot style selector
        marker_styles = ['s-', 'D-', 'o-', '*-', 'x-', 'p-']
        plotstyle = marker_styles[i % len(marker_styles)]

        # Plot
        # °C
        plt.plot(
            x_dense, y_dense,plotstyle, markerfacecolor='none', zorder=4, label=f"{condition_list[i]}M")

    plt.rcParams["font.family"] = "Times New Roman"
    # Axis labels and legend
    label = legend_label(theta_value)
    plt.xlabel(label)
    plt.ylabel(f"P({label})")
    plt.legend(loc="upper left", frameon = True)
    plt.grid(True)
    plt.xlim(0, 180)

    # Print statistics
    for i in range(len(mean)):
        print(f"i: {i}\nmean: {mean[i]:.4f}\nstdev: {stdev[i]:.4f}\nmin: {min_val[i]}\nmax: {max_val[i]}")
        print("_________________")

    # Save figure
    plt.tight_layout()
    #plt.show()
    plt.savefig(f"interpolated_Hist_{filename}.svg", dpi=300)
    plt.savefig(f"interpolated_Hist_{filename}.png", dpi=200)
    plt.close()

In [53]:
1plot_interpolated_histogram([theta_65C_01M, theta_65C_25M, theta_65C_5M, theta_65C_75M, theta_65C_1M], 1, [0.1, 0.25, 0.5, 0.75, 1.0], "5_arm_65C_12_new")

i: 0
mean: 82.0404
stdev: 36.3098
min: 1.8437
max: 178.3325
_________________
i: 1
mean: 89.0041
stdev: 37.6475
min: 1.057
max: 178.365
_________________
i: 2
mean: 95.3045
stdev: 37.5265
min: 1.6084
max: 178.8618
_________________
i: 3
mean: 97.6652
stdev: 38.4086
min: 0.4475
max: 179.8592
_________________
i: 4
mean: 97.1594
stdev: 37.0491
min: 0.6648
max: 179.4964
_________________


In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# sns.set_theme(style="ticks", palette="pastel")

# #data
# dfs = [theta_35C_01M, theta_35C_25M, theta_35C_5M, theta_35C_75M, theta_35C_1M]

# #df columns
# columns = ["0.1M", "0.25M", "0.5M", "0.75M", "1.0M"]

# all_data1 = []
# all_data2 = []
# all_data3 = []
# all_data4 = []

# #create empty dataframe
# final_df = pd.DataFrame(columns = ["conditions"])

# for i, df in enumerate(dfs):
    
#     # Extract relevant angle column
#         angles1 = df["θ1"]
#         angles2 = df["θ2"]
#         angles3 = df["θ3"]
#         angles4 = df["θ4"]

#         all_data1.extend([angles1, columns[i]])
#         all_data2.extend(angles2)

#         final_df = pd.concat([final_df, angles1], ignore_index=True)
#         final_df["conditions"] = columns[i]
# dfs = []
# for i in range(0, len(all_data), 2):
#     series1 = all_data1[i]
#     series2 = all_data2[i]
#     label = all_data1[i + 1]
    

#     df_temp = pd.DataFrame({'data1': series1, 'data2' : series2})
#     df_temp['condition'] = label
#     dfs.append(df_temp)

# final_df = pd.concat(dfs, ignore_index=True)
# # final_df.insert(1, "data2", all_data2)


# sns.boxplot(x='condition', y='data', data=final_df)


# Combined Boxplot

In [ ]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Set style and context
# sns.set_style("whitegrid")
# sns.set_context("talk", font_scale=1.2)
# # sns.set_palette("Set2")

# # Combine all DataFrames (same as your code)
# dfs = [theta_35C_01M, theta_35C_25M, theta_35C_5M, theta_35C_75M, theta_35C_1M]
# columns = ["0.1M", "0.25M", "0.5M", "0.75M", "1.0M"]

# all_dfs = []

# for i, df in enumerate(dfs):
#     temp_df = pd.concat([
#         pd.DataFrame({'angle_value': df['θ1'], 'angle_type': 'θ1'}),
#         pd.DataFrame({'angle_value': df['θ2'], 'angle_type': 'θ2'}),
#         pd.DataFrame({'angle_value': df['θ3'], 'angle_type': 'θ3'}),
#         pd.DataFrame({'angle_value': df['θ4'], 'angle_type': 'θ4'}),
#     ])
#     temp_df['condition'] = columns[i]
#     all_dfs.append(temp_df)

# final_df = pd.concat(all_dfs, ignore_index=True)
# final_df = final_df.dropna(subset=['angle_value'])

# final_df['angle_type'] = final_df['angle_type'].replace({
#     'θ1': r'$\theta_1$',
#     'θ2': r'$\theta_2$',
#     'θ3': r'$\theta_3$',
#     'θ4': r'$\theta_4$',
# })


# # Plot
# plt.figure(figsize=(10, 6))

# sns.boxplot(x = 'condition', y = 'angle_value', hue = 'angle_type', data=final_df)

# # Optional: overlay swarmplot for data points
# # sns.swarmplot(x='condition', y='angle_value', hue='angle_type', data=final_df, dodge=True, color=".25", size=2)

# # Beautify the plot
# plt.rcParams["font.family"] = "Times New Roman"
# plt.rcParams['text.usetex'] = False

# plt.title("Angle Distributions Across Conditions", fontsize=25, fontname = 'Times New Roman', pad = 20)
# plt.xlabel("Condition", fontsize=20, labelpad = 15)
# plt.ylabel("Angle Value (°)", fontsize=20, labelpad = 15)
# plt.xticks(rotation=15)
# plt.legend(title="Angle Type", bbox_to_anchor=(1.05, 1), loc='upper left')

# plt.tight_layout()

# plt.show()


# Single Boxplot

In [ ]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Set style and context
# sns.set_style("whitegrid")
# sns.set_context("talk", font_scale=1.2)
# sns.set_palette("Set2")

# # Combine all DataFrames (same as your code)
# dfs = [theta_35C_01M, theta_35C_25M, theta_35C_5M, theta_35C_75M, theta_35C_1M]
# columns = ["0.1M", "0.25M", "0.5M", "0.75M", "1.0M"]

# all_dfs = []

# for i, df in enumerate(dfs):
#     temp_df = pd.concat([
#         pd.DataFrame({'angle_value': df['θ1'], 'angle_type': 'θ1'})
#     ])
#     temp_df['condition'] = columns[i]
#     all_dfs.append(temp_df)

# final_df = pd.concat(all_dfs, ignore_index=True)
# final_df = final_df.dropna(subset=['angle_value'])

# final_df['angle_type'] = final_df['angle_type'].replace({
#     'θ1': r'$\theta_1$',
# })


# # Plot
# plt.figure(figsize=(10, 6))

# sns.boxplot(x = 'condition', y = 'angle_value', data=final_df)

# # Optional: overlay swarmplot for data points
# # sns.swarmplot(x='condition', y='angle_value', hue='angle_type', data=final_df, dodge=True, color=".25", size=2)

# # Beautify the plot
# plt.rcParams["font.family"] = "Times New Roman"
# plt.rcParams['text.usetex'] = False

# plt.title("Angle Distributions Across Conditions", fontsize=25, fontname = 'Times New Roman', pad = 20)
# plt.xlabel("Condition", fontsize=20, labelpad = 15)
# plt.ylabel("Angle Value (°)", fontsize=20, labelpad = 15)
# plt.xticks(rotation=15)
# # plt.legend(title="Angle Type", bbox_to_anchor=(1.05, 1), loc='upper left')

# plt.tight_layout()

# plt.show()


# Violin plot

In [ ]:
# plt.figure(figsize=(10, 10))

# sns.violinplot(x = 'condition', y = 'angle_value', data=final_df)

# # Optional: overlay swarmplot for data points
# # sns.swarmplot(x='condition', y='angle_value', hue='angle_type', data=final_df, dodge=True, color=".25", size=2)

# # Beautify the plot
# plt.rcParams["font.family"] = "Times New Roman"
# plt.rcParams['text.usetex'] = False

# plt.title("Angle Distributions Across Conditions", fontsize=25, fontname = 'Times New Roman', pad = 20)
# plt.xlabel("Condition", fontsize=20, labelpad = 15)
# plt.ylabel("Angle Value (°)", fontsize=20, labelpad = 15)
# plt.xticks(rotation=15)
# # plt.legend(title="Angle Type", bbox_to_anchor=(1.05, 1), loc='upper left')

# plt.tight_layout()

# plt.show()


# d_p

In [ ]:
# import numpy as np

# dataframes = traj_50_1M
# topFile = "/Users/pradnyakadam/Downloads/ACS_Manuscript/4_arm_INPUT_FILES/4m1_6NT_1_GCUAGC.top"

# avg_core_indecies, end_indecies = find_arm_indecies(topFile, 6, 4, 1, 2)
# com_dfs = find_center_of_mass(dataframes, avg_core_indecies)

# # print(com_dfs)

# #defines the starting point as the coordinates position associated with the particular timestamp
# #refernce comment above for more info.
# start_pt = np.array([com_dfs.iloc[0, 0], com_dfs.iloc[0, 1], com_dfs.iloc[0, 2]])


# #defining vector 1 end point based on the provided index
# end_pt_1 = np.array([float(df.iloc[0, 0]), float(df.iloc[0, 1]), float(df.iloc[0, 2])])


# # #defining vector 2 end point based on the other provided index
# end_pt_2 = np.array([float(df.iloc[78, 0]), float(df.iloc[78, 1]), float(df.iloc[78, 2])])

# # #defining vector 2 end point based on the other provided index
# end_pt_3 = np.array([float(df.iloc[117, 0]), float(df.iloc[117, 1]), float(df.iloc[117, 2])])

# end_pt_4 = np.array([float(df.iloc[39, 0]), float(df.iloc[39, 1]), float(df.iloc[39, 2])])



# # #defining each vector to represent an arm of the nanostar
# v1 = end_pt_1 - start_pt
# v2 = end_pt_2 - start_pt
# v3 = end_pt_3 - start_pt
# v4 = end_pt_4 - start_pt

# end_pt_1
    

In [ ]:
traj_50_1M